In [11]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
from pandasql import sqldf
import requests as req
import pandas as pd
import numpy as np
import sqlite3
import time
import os
import re

In [2]:
NOW     = datetime.now()
Y, M, D = NOW.year, NOW.month, NOW.day

In [3]:
whole_page = ""

start_time = time.time()

for idx in range(5):
    url = f'https://store.steampowered.com/search/?specials=1&filter=topsellers&page={idx}'
    res = req.get(url)
    whole_page += res.text

time.time() - start_time

2.700230836868286

In [4]:
start_time = time.time()
soup       = bs(whole_page, 'html.parser')
sales      = soup.select('div#search_resultsRows > a') 
time.time() - start_time

10.572810411453247

In [5]:
dbpath = '../db/test.db'
conn   = sqlite3.connect(dbpath)
cursor = conn.cursor()

In [6]:
query = """
CREATE TABLE IF NOT EXISTS steamsale(
id INTEGER PRIMARY KEY AUTOINCREMENT,
appid INTEGER NOT NULL,
percent INTEGER NOT NULL,
original INTEGER NOT NULL,
discounted INTEGER NOT NULL,
date TEXT NOT NULL);"""

cursor.execute(query)

In [7]:
def preprop(string, dtype = 'discount'): 
    
    if dtype == 'discount': string = string.replace(' ', '').split('₩')[-1]
    string = string.replace(',', '').replace('₩', '')
    
    return int(string)
    

sale_list = []
for idx, sale in enumerate(sales):
    try:
        appid      = sale['data-ds-appid']

        percent    = sale.select('.search_discount > span')[0].text
        percent    = int(percent.replace('-', '').replace('%', ''))

        original   = preprop(sale.select('strike')[0].text)
        discounted = preprop(sale.select(".discounted")[0].text)

        today = f'{Y}{str(M).zfill(2)}{str(D).zfill(2)}'
        query = f"""
                INSERT INTO steamsale
                (appid, percent, original, discounted, date)
                VALUES({appid}, {percent}, {original}, {discounted}, {today});
                """

        cursor.execute(query)
              
    except Exception as e: print(e)

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


In [8]:
with conn:
    with open('../db/test_backup.sql', 'w') as f:
        for line in conn.iterdump():
            f.write('%s\n' % line)
        print('Completed')

Completed


In [9]:

## DB에 있는 데이터를 조회하는 함수
## 고급 쿼리 부분은 좀 더 구현해야 한다.
def search_table(cursor, table_name, columns = '*', **kwargs):

    sorting_col = kwargs['sorting_col'] if 'sorting_col' in kwargs.keys() else 'original'
    conditions  = kwargs['conditions']  if 'conditions' in kwargs.keys() else None
    how_many    = kwargs['how_many']    if 'how_many' in kwargs.keys() else 1
    reverse     = kwargs['desc']         if 'desc' in kwargs.keys() else False
 
    col_indexes = {k : v for v, k in enumerate(['id', 'appid', 'percent', 'original', 
                                                 'discounted', 'date'])}
    col_splited = columns.replace(' ','').split(',')
    col_indexes = col_indexes if columns == '*' else \
                  {k : v for v, k in enumerate(col_splited)}
    assert sorting_col in columns, f'''[ERR.Col-0001] 선택하신 조건에 맞는 컬럼이 존재하지 않아 선택 하신 옵션으로 정렬 할 수 없었습니다. {columns}에서 골라 주십시오.'''
    
    
    if conditions == None:
        query = f"""
                    SELECT {columns} FROM {table_name}
                """
    else:
        ## WHERE 문으로 찾을 column, 조건, 데이터를 받아 조회해준다.
        conditions = np.array(conditions)
        
        if len(conditions) == 3:
            col, cond, data = conditions

            symbols = ['>', '<', '!=', '=', '>=', '<=', 'IN']
            assert cond in symbols, f'[ERR.Co-0001] 조건이 올바르지 않습니다. {symbols}에서 선택해 넣어주십시오.'
    
            
            query = f"""
                        SELECT {columns} FROM {table_name}
                        WHERE {col} {cond} {data};
                    """
            
        elif len(conditions) == 2:
            col, data = conditions
            query = f"""
                        SELECT {columns} FROM {table_name}
                        WHERE {col}={data};
                    """
            
            q = cursor.execute('select * from steamsale')
    
            
    cursor.execute(query)
    
    ## how_many가 0을 포함한 음의 정수가 된다면 모든 데이터를 조회해준다.
    col_index = col_indexes[sorting_col]
    return sorted(cursor.fetchmany(how_many), key = lambda x: (x[col_index], x[0]), 
                                              reverse = reverse) 

In [16]:
search_table(cursor, 'steamsale', columns = 'appid, original, discounted', 
             desc = True, how_many = 0, sorting_col = 'original', 
             conditions = ['original', '>=', '65000'])

conn.close()

In [17]:
ROOT_PATH  = '/config/workspace/project'
DB_PATH     = f'{ROOT_PATH}/DoveNest/informations/db'

In [18]:
class salesDB:

    def __init__(self, table_name, db_name):

        self.table_name = table_name
        self.db_name    = db_name
        
        ## DataBase 연결
        self.connect_db()


    ## DB와 연결시켜주는 함수
    def connect_db(self):

        dbpath = f'{DB_PATH}/{self.db_name}.db'
        self.conn   = sqlite3.connect(dbpath, check_same_thread = False)
        self.cursor = self.conn.cursor()


    ## DB에 있는 데이터를 조회하는 함수 고급 쿼리 부분은 좀 더 구현해야 한다.
    def search_table(self, columns = '*', **kwargs):

        ## keyword argument 값에서 데이터가 없는 경우 기본값 지정

        ## 이부분도 너무 킹받게 짜졌다,, 안되는거 그냥 덕지덕지 수정했더니...
        sorting_col = kwargs['sorting_col'] if 'sorting_col' in kwargs.keys() else columns
        sorting_col = sorting_col if sorting_col != '*' else  \
                    ('appid' if type(sorting_col) != list else sorting_col[0])

        conditions  = kwargs['conditions']  if 'conditions' in kwargs.keys() else None
        how_many    = kwargs['how_many']    if 'how_many' in kwargs.keys() else 1
        reverse     = kwargs['desc']        if 'desc' in kwargs.keys() else False

        
        col_indexes = {k : v for v, k in enumerate(['id', 'appid', 'percent', 'original', 
                                                    'discounted', 'date'])}

        ## 코드가 너무 킹받게 짜졌다..
        col_indexes = col_indexes if columns == '*' else \
                    ({k : v for v, k in enumerate(columns)} \
                    if type(columns) == list else {columns : 0})

        col_keys = [col for col in col_indexes.keys()]
        try:
            print(sorting_col, col_keys)
            assert sorting_col in col_keys, f'''\n[ERR.DB.Co-0001] 선택하신 조건에 맞는 컬럼이 존재하지 않아 선택 하신 옵션으로 정렬 할 수 없었습니다. \
                                                {col_keys}에서 골라 주십시오.'''

            columns = ', '.join(columns) if type(columns) == list else columns


            ## 데이터 조회할 때 그 어떤 조건도 없는 경우 그냥 테이블에서 컬럼만 받아 사용
            if conditions == None:
                query = f"""
                            SELECT DISTINCT {columns} FROM {self.table_name}
                        """
            
            else:
                ## 고급 쿼리에 사용할 거
                conditions = np.array(conditions)

                ## WHERE 문으로 찾을 column, 조건, 데이터를 받아 조회해준다.
                if len(conditions) == 3:
                    col, cond, data = conditions

                    symbols = ['>', '<', '!=', '=', '>=', '<=', 'IN']
                    assert cond in symbols, f'\n[ERR.DB.Co-0002] 조건이 올바르지 않습니다. {symbols}에서 선택해 넣어주십시오.'
            
                    
                    query = f"""
                                SELECT DISTINCT {columns} FROM {self.table_name}
                                WHERE {col} {cond} {data};
                            """
                
                ## 데이터를 찾을 column과 찾을 data만 있는 경우
                ## 기본값으로 동일한 데이터만 찾도록 지정해주었다.
                elif len(conditions) == 2:
                    col, data = conditions
                    query = f"""
                                SELECT DISTINCT {columns} FROM {self.table_name}
                                WHERE {col}={data};
                            """

            self.cursor.execute(query)
            
            ## 데이터 정렬에 사용할 인덱스 값들.
            col_index = col_indexes[sorting_col]

        except Exception as e:
            print(f'[ERR.DB.Q-0001] 쿼리에 문제가 발생하였습니다. 확인 후 수정바랍니다. {e}')
            query     = f'SELECT * FROM {self.table_name}'
            col_index = 0

        ## how_many가 0을 포함한 음의 정수가 된다면 모든 데이터를 조회해준다.
        return sorted(self.cursor.fetchmany(how_many), 
                      key = lambda x: x[col_index], reverse = reverse) 


In [21]:
DB = salesDB(table_name = 'saleinfo', db_name = 'game_informations_')

OperationalError: unable to open database file